In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [10]:
fake=pd.read_csv("Fake.csv")
true=pd.read_csv("True.csv")

In [11]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [13]:
fake['class']=0
true['class']=1

In [14]:
data=pd.concat([fake,true],axis=0)

In [15]:
data.sample(10)

,title,text,subject,date,class
23042,It’s Official: Trump is POTUS 45,Today Donald J. Trump was sworn in as the 45th...,Middle-east,"January 20, 2017",0
18025,Spain's Colonial calls board meeting for Monda...,MADRID (Reuters) - Spanish real estate firm In...,worldnews,"October 8, 2017",1
10536,New Jersey Governor Christie says he urged Tru...,(Reuters) - New Jersey Governor Chris Christie...,politicsNews,"March 3, 2016",1
15731,LYING WHITE HOUSE PRESS SECRETARY: “OBAMA HAS ...,It s interesting that Josh Earnest still carri...,politics,"May 7, 2015",0
5329,Exclusive: White House delivered EU-skeptic me...,BERLIN (Reuters) - In the week before U.S. Vic...,politicsNews,"February 21, 2017",1
5752,Vatican worried about Trump immigration order,VATICAN CITY (Reuters) - The Vatican said on W...,politicsNews,"February 1, 2017",1
8944,Trump Goes Full Birther — On Ted Cruz,Donald Trump has revived the birther conspirac...,News,"January 6, 2016",0
21108,"U.N. chief condemns N.Korea nuclear test, says...",(Reuters) - United Nations Secretary-General A...,worldnews,"September 3, 2017",1
6678,Colbert Outs Republicans As The True Weirdos ...,Late Show host Stephen Colbert hilariously too...,News,"April 27, 2016",0
7750,Final Trump-Clinton debate draws nearly 72 mil...,(Reuters) - An estimated 71.6 million people w...,politicsNews,"October 20, 2016",1


In [17]:
data=data.drop(["title","subject","date"],axis=1)

In [18]:
data.reset_index(inplace=True)

In [19]:
data.drop(['index'],axis=1,inplace=True)

In [20]:
data.sample(5)

,text,class
28748,(Reuters) - Highlights of the day for U.S. Pre...,1
39183,VIENNA (Reuters) - The fallout from the Harvey...,1
4564,"The city of Flint, Michigan is suffering throu...",0
32909,SAN FRANCISCO (Reuters) - Facebook Inc (FB.O) ...,1
39019,NEW YORK (Reuters) - Former Soviet leader Mikh...,1


In [28]:
def clean_text(text):
    text=text.lower()
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n","",text)
    text = re.sub(r"\w*\d\w*", "",text)
    return text
    
    
    
    
    

In [29]:
data["text"] = data["text"].apply(clean_text)


In [30]:
x=data['text']
y=data['class']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42)


In [31]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_text=vectorizer.transform(xtest)


In [32]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [39]:
prediction = lr.predict(xv_text)
lr.score(xv_text,ytest)

0.9858351893095768

In [40]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [42]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")


['lr_model.jb']